In [ ]:
using PowerModels
using Ipopt
using ForwardDiff
using OrdinaryDiffEq
using NLsolve

display(HTML("<style>.container { width:100% !important; }</style>"))

data = PowerModels.parse_file("C:/Users/Ewa/Downloads/Twogen2bus.m")
pf_result = run_ac_pf(data ,Ipopt.Optimizer)
#PowerModels.print_summary(pf_result["solution"])

# Load flow results

# bus results
va1 =pf_result["solution"]["bus"]["1"]["va"]
va2 =pf_result["solution"]["bus"]["2"]["va"]
vm1 =pf_result["solution"]["bus"]["1"]["vm"]
vm2 =pf_result["solution"]["bus"]["2"]["vm"];

# Voltage at bus 1
V_tR = vm1*cos(va1)
V_tI = vm1*sin(va1)


# Voltage at bus 2
V_tR1 = vm2*cos(va2)
V_tI1 = vm2*sin(va2)

# generator results
pg = pf_result["solution"]["gen"]["1"]["pg"]
qg =pf_result["solution"]["gen"]["1"]["qg"]

# branch results
p_ref =pf_result["solution"]["branch"]["1"]["pf"]
q_ref =pf_result["solution"]["branch"]["1"]["qf"]
p_ref1 =pf_result["solution"]["branch"]["1"]["pt"]
q_ref1 =pf_result["solution"]["branch"]["1"]["qt"];

In [ ]:
# Define the parameters
lf =0.08
rf = 0.003
cf = 0.074
rt = 0.2
lt = 0.01
rl = 0.01

# Line parameters
# What values should be used?
ick_d =  0.01
ick_q = 0.01
icj_d = 0.02
icj_q = 0.02
c_lk = 0.02
l_lk = 0.1
r_lk = 0.01
c_lj = 0.02
l_lj = 0.1
r_lj = 0.01
r_jk = 0.01
l_jk = 0.02


kp_pll = 0.4
ki_pll = 4.69

ωb = 2*pi*50.0
ωb1 = 2*pi*50.0

Rp = 0.1
Rq =0.1
ωz = 2*pi*5.0
ωz1 = 2*pi*5.0
ωg = 1.0

kvp = 0.52
kvi = 1.16
kvf = 1.0
rv = 0.0
lv =0.2
kip = 0.74
kii = 1.19
kif = 1.0

ω_ref = 1.0
v_ref = 0.8
ωn = 2*pi*50.0
ωg =1.0
ω_star = ωg


    function system!(F,x)
###Grid following equations
    #𝜕ed/𝜕t
    F[1] = (ωb / cf * x[3] - ωb / cf * x[5] + ωb * ωg * x[2])
    #𝜕eq/𝜕t
    F[2] = (ωb / cf * x[4] - ωb / cf * x[6] - ωb* ωg *x[1])
    #𝜕isd/𝜕t
    F[3] = (ωb / lf * (  kip * ( (kvp * (((v_ref + Rq*(q_ref1 - x[11]))- rv * x[5] + ((ωn + kp_pll*(x[1]*sin(x[7] - x[8]) + x[2]*cos(x[8] -x[7])) + ki_pll*x[9]) + Rp*(p_ref1 - x[10]))* lv * x[6]) - x[1]) + kvi * x[12]
    - cf *   ((ωn + kp_pll*(x[1]*sin(x[7] - x[8]) + x[2]*cos(x[8] -x[7])) + ki_pll*x[9]) + Rp*(p_ref1 - x[10]))* x[2] + kif * x[5]) - x[3]) + kii * x[14] - ((ωn + kp_pll*(x[1]*sin(x[7] - x[8]) + x[2]*cos(x[8] -x[7])) + ki_pll*x[9]) + Rp*(p_ref1 - x[10])) * lf * x[4] + kvf *x[1])  - ωb / lf *x[1] - ωb * rf / lf * x[3] + ωb * ωg * x[4])
    #𝜕isq/𝜕t
    F[4] = (ωb / lf * ((kip * ((kvp * ((-rv * x[6] -  ((ωn + kp_pll*(x[1]*sin(x[7] - x[8]) + x[2]*cos(x[8] -x[7])) + ki_pll*x[9]) + Rp*(p_ref1 - x[10])) * lv * x[5]) - x[2]) + kvi * x[13] + cf * ((ωn + kp_pll*(x[1]*sin(x[7] - x[8]) + x[2]*cos(x[8] -x[7])) + ki_pll*x[9]) + Rp*(p_ref1 - x[10])) *x[1] + kif * x[6]) - x[4])
    + kii * x[15] +  ((ωn + kp_pll*(x[1]*sin(x[7] - x[8]) + x[2]*cos(x[8] -x[7])) + ki_pll*x[9]) + Rp*(p_ref1 - x[10]))* lf * x[3] + kvf * x[2] )) - ωb / lf * x[2] - ωb * rf / lf * x[4] - ωb * ωg * x[3])
    #𝜕igd/𝜕t
    F[5] = (ωb / lt *x[1] - ωb / lt * (rl*x[6]) - ωb * rt / lt * x[5] + ωb * ωg * x[6])
    #𝜕igq/𝜕t
    F[6] = (ωb / lt * x[2] + ωb / lt *(rl*x[5])  - ωb * rt / lt * x[6] - ωb * ωg * x[5])
    # PLL states
    #𝜕δc/𝜕t
    F[7] = (((ωn + kp_pll*(x[1]*sin(x[7] - x[8]) + x[2]*cos(x[8] -x[7])) + ki_pll*x[9]) + Rp*(p_ref1 - x[10]))- ωg)*ωb
    #𝜕δpll/𝜕t
    F[8] = (ωn - ωg + kp_pll*(x[1]*sin(x[7] - x[8]) + x[2]*cos(x[8] -x[7])) + ki_pll*x[9])*ωb
    #𝜕ϵ_pll/𝜕t
    F[9] = x[1]*sin(x[7] - x[8]) + x[2]*cos(x[8] -x[7])
    # Outer control model - virtual inertia
    #𝜕pm/𝜕t
     F[10] = (ωz*x[6]*x[2] + ωz*x[5]*x[1]- ωz*x[10])
    #𝜕qm/𝜕t
    F[11] = (-ωz * x[6] *x[1] + ωz * x[5] * x[2] - ωz * x[11])
    # Voltage control
    #PI Integrator (internal state)
    #𝜕ξ_d/𝜕t
    F[12] = (((v_ref + Rq*(q_ref1 - x[11]))  - rv * x[5] + ((ωn + kp_pll*(x[1]*sin(x[7] - x[8]) + x[2]*cos(x[8] -x[7])) + ki_pll*x[9]) + Rp*(p_ref1 - x[10]))* lv * x[6]) -x[1])
     #𝜕ξ_q/𝜕t
     F[13] = (-rv * x[6] - ((ωn + kp_pll*(x[1]*sin(x[7] - x[8]) + x[2]*cos(x[8] -x[7])) + ki_pll*x[9]) + Rp*(p_ref1 - x[10])*lv*x[5]) - x[2])
    # Current control
    # PI integrator (internal state)
    #𝜕γ_d/𝜕t
     F[14] = (kvp * (((v_ref + Rq*(q_ref1 - x[11])) - rv * x[5] +  ((ωn + kp_pll*(x[1]*sin(x[7] - x[8]) + x[2]*cos(x[8] -x[7])) + ki_pll*x[9]) + Rp*(p_ref1 - x[10]))* lv * x[6]) -x[1]) + kvi * x[12] - cf *  ((ωn + kp_pll*(x[1]*sin(x[7] - x[8]) + x[2]*cos(x[8] -x[7])) + ki_pll*x[9]) + Rp*(p_ref1 - x[10])) * x[2] + kif * x[5]) - x[3]
    #𝜕γ_q/𝜕t
    F[15] = (kvp * ((-rv * x[6] -  ((ωn + kp_pll*(x[1]*sin(x[7] - x[8]) + x[2]*cos(x[8] -x[7])) + ki_pll*x[9]) + Rp*(p_ref1 - x[10])) * lv * x[5]) - x[2]) + kvi * x[13] + cf * ((ωn + kp_pll*(x[1]*sin(x[7] - x[8]) + x[2]*cos(x[8] -x[7])) + ki_pll*x[9]) + Rp*(p_ref1 - x[10])) *x[1] + kif * x[6]) - x[4]
end
# Initial Conditions
     initial =  [ V_tR, #egd
            V_tI, #egq
            p_ref1/V_tR, #isd
            0.0, #isq
            p_ref1/V_tR, #igd
            0.0,#igq
            0.0, #δc
            0.0, #δpll
            0.0, #ϵ_pll
            p_ref1, #pm
            q_ref1, #qm
            0.0, #ξ_d
            0.0, #ξ_q
            0.0, #γ_d
            0.0, #γ_d
    ]


         sys_solve = NLsolve.nlsolve(
                     system!,
                     initial,
                     xtol = :1e-9,
                     ftol = :1e-9,
                     method = :trust_region,
                    ) #Solve using initial guess x0
                      print(sys_solve)